# Cars: Grouping and Aggregation

## Imports

In [1]:
import pandas as pd
import numpy as np
from altair import load_dataset

## Dataset

In [2]:
cars = load_dataset('cars')

In [3]:
cars.head()

,Acceleration,Cylinders,Displacement,Horsepower,Miles_per_Gallon,Name,Origin,Weight_in_lbs,Year
0,12.0,8,307.0,130.0,18.0,chevrolet chevelle malibu,USA,3504,1970-01-01
1,11.5,8,350.0,165.0,15.0,buick skylark 320,USA,3693,1970-01-01
2,11.0,8,318.0,150.0,18.0,plymouth satellite,USA,3436,1970-01-01
3,12.0,8,304.0,150.0,16.0,amc rebel sst,USA,3433,1970-01-01
4,10.5,8,302.0,140.0,17.0,ford torino,USA,3449,1970-01-01


In [4]:
len(cars)

406

## Grouped mean

Compute the average of `Acceleration`, `Displacement` and `Horsepower`, grouped by `Origin`:

In [5]:
g1 = cars.groupby('Origin')['Acceleration', 'Displacement', 'Horsepower'].mean()
g1.head()

,Acceleration,Displacement,Horsepower
Origin,,,
Europe,16.821918,109.465753,81.000000
Japan,16.172152,102.708861,79.835443
USA,14.942520,247.935039,119.900000


In [6]:
assert list(g1.columns)==['Acceleration', 'Displacement', 'Horsepower']
assert list(g1.index)==['Europe', 'Japan', 'USA']

Find the maximum `Acceleration` and `Displacement` grouped by `Origin` and `Cylinders`:

In [7]:
g2 = cars.groupby(['Origin', 'Cylinders'])['Acceleration', 'Displacement'].max()

In [8]:
assert list(g2.columns)==['Acceleration', 'Displacement']
assert [list(i) for i in list(g2.index.levels)]==[['Europe', 'Japan', 'USA'], [3, 4, 5, 6, 8]]

## Grouped counts

Compute the number of cars, grouped by `Year` and `Cylinders` and unstack the result:

In [9]:
g3 = cars.groupby('Year')['Cylinders'].aggregate({3:'count', 4:'count', 5:'count', 6:'count', 8:'count'})
g3=g3.loc[:,[3, 4, 5, 6, 8]]

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  """Entry point for launching an IPython kernel.


In [10]:
assert list(g3.columns)==[3, 4, 5, 6, 8]
assert list(g3.index)==['1970-01-01', '1971-01-01', '1972-01-01', '1973-01-01',
                        '1974-01-01', '1975-01-01', '1976-01-01', '1977-01-01', 
                        '1978-01-01', '1979-01-01', '1980-01-01', '1982-01-01']

## Multiple aggregations

Compute the min and max `Miles_per_Gallon` grouped by `Origin`:

In [11]:
g4 = cars.groupby('Origin')['Miles_per_Gallon'].aggregate({'max':'max', 'min':'min'})
g4=g4.loc[:,['max', 'min']]
g4.head()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  """Entry point for launching an IPython kernel.


,max,min
Origin,,
Europe,44.3,16.2
Japan,46.6,18.0
USA,39.0,9.0


In [12]:
assert set(g4.columns)==set(['min', 'max'])
assert list(g4.index)==['Europe', 'Japan', 'USA']

## Custom aggregation function

Compute the range of cylinders (`range` = max - min) grouped by `Origin` and name the resulting column `range`:

In [13]:
rang = lambda x: x.max() - x.min()
g5 = cars.groupby('Origin')['Cylinders'].aggregate({'range':rang})

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  


In [14]:
assert list(g5.columns)==['range']
assert list(g5.index)==['Europe', 'Japan', 'USA']

## Group filtering

Compute the average acceleration, grouped by `Year` and `Origin`, only including groups with a max number of Cylinders less than 6. Unstack the `Origin` level of the resulting hierarchical row index:

In [15]:
def filt(x):
    return x['Cylinders'].max() < 6
g6 = cars.groupby(['Origin', 'Year']).filter(filt)
g6 = g6.groupby(['Year', 'Origin'])[['Acceleration']].mean()
g6=g6.unstack()

In [16]:
assert [list(i) for i in list(g6.columns.levels)]==[['Acceleration'], ['Europe', 'Japan']]
assert g6.index.name=='Year'
assert len(g6)==10

## Grouped z-scores

Here is the average `Miles_per_Gallon`, grouped by `Year`:

In [17]:
cars.groupby(['Year'])['Miles_per_Gallon'].mean()

Year
1970-01-01    17.689655
1971-01-01    21.250000
1972-01-01    18.714286
1973-01-01    17.100000
1974-01-01    22.703704
1975-01-01    20.266667
1976-01-01    21.573529
1977-01-01    23.375000
1978-01-01    24.061111
1979-01-01    25.093103
1980-01-01    33.696552
1982-01-01    31.045000
Name: Miles_per_Gallon, dtype: float64

Replace the `Miles_per_Gallon` values by the [z-score](https://en.wikipedia.org/wiki/Standard_score) of that value relative to its group for each `Year`:

In [18]:
zscore_mpg = cars.groupby(['Year'])[['Miles_per_Gallon']].transform(lambda x: (x - x.mean())/x.std())

Show that the average z-scores for `Miles_per_Gallon` grouped by year are all zero:

In [19]:
zscore_mpg.mean()

Miles_per_Gallon   -6.694812e-18
dtype: float64

Show that the standard deviation of the z-scores for `Miles_per_Gallon` grouped by year are all 1.0:

In [20]:
zscore_mpg.std()

Miles_per_Gallon    0.986049
dtype: float64

## Grouped missing value replacement

## Introduce missing values

Let's introduce some missing values into the `Cylinders` column:

In [21]:
cars['Cylinders'] = np.where(np.random.rand(len(cars)) > 0.8, np.nan, cars['Cylinders'])
cars.Cylinders

0      8.0
1      8.0
2      8.0
3      8.0
4      8.0
5      8.0
6      8.0
7      8.0
8      NaN
9      8.0
10     NaN
11     8.0
12     NaN
13     8.0
14     8.0
15     8.0
16     8.0
17     8.0
18     8.0
19     8.0
20     4.0
21     NaN
22     NaN
23     6.0
24     4.0
25     4.0
26     4.0
27     NaN
28     4.0
29     NaN
      ... 
376    4.0
377    4.0
378    4.0
379    NaN
380    NaN
381    NaN
382    4.0
383    4.0
384    NaN
385    4.0
386    4.0
387    4.0
388    4.0
389    4.0
390    4.0
391    4.0
392    4.0
393    4.0
394    NaN
395    6.0
396    4.0
397    6.0
398    4.0
399    4.0
400    4.0
401    NaN
402    NaN
403    4.0
404    4.0
405    4.0
Name: Cylinders, Length: 406, dtype: float64

Here are the average number of Cylinders, grouped by `Year`:

In [22]:
cars.groupby(['Year'])['Cylinders'].mean()

Year
1970-01-01    7.142857
1971-01-01    5.583333
1972-01-01    5.739130
1973-01-01    6.517241
1974-01-01    5.700000
1975-01-01    5.833333
1976-01-01    5.724138
1977-01-01    5.333333
1978-01-01    5.218750
1979-01-01    5.772727
1980-01-01    4.173913
1982-01-01    4.425532
Name: Cylinders, dtype: float64

Replace the missing values in `Cylinders` by the group average (grouped by `Year`):

In [23]:
dirtycars,cleancars=cars,cars
def replacenanwithgroupaverage(x):
    val = cars.groupby(['Year'])['Cylinders'].mean().loc[x.name]
    x.fillna(val, inplace=True)
    return x
cleancars['Cylinders']= dirtycars.groupby(['Year'])[['Cylinders']].apply(replacenanwithgroupaverage)

By replacing missing values with group averages, the grouped averages remain unchanged:

In [24]:
assert list(cleancars.groupby('Year')['Cylinders'].mean())==list(dirtycars.groupby('Year')['Cylinders'].mean())